ResNet18 모델 정의 및 인스턴스 초기화

In [50]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os

#Basic Block 정의
class BasicBlock(nn.Module):
  def __init__(self, in_planes, planes, stride = 1):
    super(BasicBlock, self).__init__()

    self.conv1 = nn.Conv2d(in_planes, planes, kernel_size = 3, stride = stride, padding = 1,  bias = False)
    self.bn1 = nn.BatchNorm2d(planes)

    self.conv2 = nn.Conv2d(planes, planes, kernel_size = 3, stride = 1, padding = 1, bias = False)
    self.bn2 = nn.BatchNorm2d(planes)
    #nn.Sequential은 다른 Module들을 포함하는 Module로, 그 Module들을 순차적으로 적용하여 출력을 생성한다.
    #nn.Sequential(x)이면 x가 그대로 들어간다.
    self.shortcut = nn.Sequential()
    if stride != 1:
      self.shortcut = nn.Sequential(
          nn.Conv2d(in_planes, planes, kernel_size = 1, stride = stride, bias = False),
          nn.BatchNorm2d(planes)
      )
  def forward(self, x):
    out = F.relu(self.bn1(self.conv1(x)))
    out = self.bn2(self.conv2(out))
    out += self.shortcut(x)
    out = F.relu(out)
    return out
#ResNet 정의
class ResNet(nn.Module):
  def __init__(self, block, num_block, num_classes = 10):
    super(ResNet, self).__init__()
    self.in_planes = 64
    #64개의 3x3 필터를 사용
    self.conv1 = nn.Conv2d(3, 64, kernel_size = 3, stride = 1, padding = 1, bias = False)
    self.bn1 = nn.BatchNorm2d(64)
    self.layer1 = self._make_layer(block, 64, num_block[0], stride = 1)
    self.layer2 = self._make_layer(block, 128, num_block[1], stride = 2)
    self.layer3 = self._make_layer(block, 256, num_block[2], stride = 2)
    self.layer4 = self._make_layer(block, 512, num_block[3], stride = 2)
    self.linear = nn.Linear(512, num_classes)
  def _make_layer(self, block, planes, num_block, stride):
    strides = [stride] + [1]*(num_block-1)
    layers = []
    for stride in strides:
      layers.append(block(self.in_planes, planes, stride))
      self.in_planes = planes    
    return nn.Sequential(*layers)
  def forward(self, x):
    out = F.relu(self.bn1(self.conv1(x)))
    out = self.layer1(out)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    out = F.avg_pool2d(out, 4)
    out = out.view(out.size(0), -1)
    out = self.linear(out)
    return out

#ResNey18함수 정의
def ResNet18():
  return ResNet(BasicBlock, [2, 2, 2, 2])





데이터셋 다운로드 및 불러오기

In [51]:
import torchvision
import torchvision.transforms as transforms

transform_train = transforms.Compose([
                                      transforms.RandomCrop(32, padding =4),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
])

transform_test = transforms.Compose([
                                     transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
test_dataset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 128, shuffle = True, num_workers = 4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 100, shuffle = False, num_workers = 4)

Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


환경 설정 및 학습 함수 정의

In [55]:
device = 'cuda'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = learning_rate, momentum = 0.9, weight_decay = 0.0002)

def train(epoch):
  print('\n [Train Epoch : %d]' % epoch)

  net.train()
  train_loss = 0
  correct = 0
  total = 0

  for batch_idx, (inputs, targets) in enumerate(train_loader):
    inputs, targets = inputs.to(device), targets.to(device)
    optimizer.zero_grad()

    benign_outputs = net(inputs)
    loss = criterion(benign_outputs, targets)
    loss.backward()

    optimizer.step()
    train_loss += loss.item()

    _, predicted = benign_outputs.max(1)

    total += targets.size(0)
    correct += predicted.eq(targets).sum().item()

    if batch_idx % 100 == 0:
      print('\n Current batch:', str(batch_idx))
      print('Current benign train accuracy: ', str(predicted.eq(targets).sum().item()/targets.size(0)))
      print('Current benign train loss : ', loss.item())
  print('\n Total benign train accuracy : ', 100.*correct/total)
  print('Total benign train loss : ', train_loss)

def test(epoch):
  print('\n [Test Epoch : %d]' %epoch)
  net.eval()
  loss = 0
  correct = 0
  total = 0

  for batch_idx, (inputs, targets) in enumerate(test_loader):
    inputs, targets = inputs.to(device), targets.to(device)
    total += targets.size(0)

    outputs = net(inputs)
    loss += criterion(outputs, targets).item()

    _, predicted = outputs.max(1)
    correct += predicted.eq(targets).sum().item()
  
  print('\n Test accuracy:', 100. * correct / total)
  print('Test average loss:', loss/total)

  state = {'net':net.state_dict()}

  if not os.path.isdir('checkpoint'):
    os.mkdir('checkpoint')
  torch.save(state, './checkpoint/' + file_name)
  print('Model Saved!')

def adjust_learning_rate(optimizer, epoch):
  lr = learning_rate
  if epoch >= 100:
    lr /= 10
  if epoch >= 150:
    lr /= 10
  for param_group in optimizer.param_groups:
    param_group['lr'] = lr



In [56]:
for epoch in range(0, 20):
  adjust_learning_rate(optimizer, epoch)
  train(epoch)
  test(epoch)


 [Train Epoch : 0]


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



 Current batch: 0
Current benign train accuracy:  0.125
Current benign train loss :  2.3161888122558594

 Current batch: 100
Current benign train accuracy:  0.2421875
Current benign train loss :  1.8951737880706787

 Current batch: 200
Current benign train accuracy:  0.3203125
Current benign train loss :  1.8026175498962402

 Current batch: 300
Current benign train accuracy:  0.3515625
Current benign train loss :  1.7564424276351929

 Total benign train accuracy :  31.058
Total benign train loss :  744.8381662368774

 [Test Epoch : 0]

 Test accuracy: 36.88
Test average loss: 0.017830971002578737
Model Saved!

 [Train Epoch : 1]

 Current batch: 0
Current benign train accuracy:  0.4765625
Current benign train loss :  1.3857836723327637

 Current batch: 100
Current benign train accuracy:  0.4453125
Current benign train loss :  1.4489315748214722

 Current batch: 200
Current benign train accuracy:  0.46875
Current benign train loss :  1.3605846166610718

 Current batch: 300
Current beni